ARMA Models

In [ ]:
import inspect
import time
import warnings

import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import seaborn as sns
from IPython.display import VimeoVideo
from pymongo import MongoClient
from sklearn.metrics import mean_absolute_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA

warnings.filterwarnings("ignore")

Prepare Data

Import

In [ ]:
#Complete to the create a client to connect to the MongoDB server, assigns the "air-quality" database to db, and assigned the "nairobi" connection to nairobi
client = MongoClient(host='localhost', port=27017)
db = client["air-quality"]
nairobi = db["nairobi"]


In [ ]:
#Change your wrangle function so that it has a resample_rule argument that allows the user to change the resampling interval. The argument default should be "1H"
def wrangle(collection, resample_rule="1H"):

    results = collection.find(
        {"metadata.site": 29, "metadata.measurement": "P2"},
        projection={"P2": 1, "timestamp": 1, "_id": 0},
    )

    # Read results into DataFrame
    df = pd.DataFrame(list(results)).set_index("timestamp")

    # Localize timezone
    df.index = df.index.tz_localize("UTC").tz_convert("Africa/Nairobi")

    # Remove outliers
    df = df[df["P2"] < 500]

    # Resample and forward-fill
    y = df["P2"].resample(resample_rule).mean().fillna(method="ffill")

    return y

In [ ]:
#Use your wrangle function to read the data from the nairobi collection into the Series y
y=wrangle(nairobi)
y.head()

Explore

In [ ]:
#Create an ACF plot for the data in y. Be sure to label the x-axis as "Lag [hours]" and the y-axis as "Correlation Coefficient"
fig, ax = plt.subplots(figsize=(15, 6))
plot_acf(y, ax=ax)
plt.xlabel("Lag [hours]")
plt.ylabel("Correlation Coefficient");

In [ ]:
#Create an PACF plot for the data in y. Be sure to label the x-axis as "Lag [hours]" and the y-axis as "Correlation Coefficient"
fig, ax = plt.subplots(figsize=(15, 6))
plot_pacf(y, ax=ax)
plt.xlabel("Lag [hours]")
plt.ylabel("Correlation Coefficient");

Split

In [ ]:
#Create a training set y_train that contains only readings from October 2018, and a test set y_test that contains readings from November 1, 2018.
y_train = y["2018-10-01":"2018-10-31"]
y_test = y["2018-11-01":"2018-11-01"]

Build Model

Baseline

In [ ]:
#Calculate the baseline mean absolute error for your model.
y_pred_baseline = [y_train.mean()]*len(y_train)
mae_baseline = mean_absolute_error(y_train, y_pred_baseline)
print("Mean P2 Reading:", round(y_train.mean(), 2))
print("Baseline MAE:", round(mae_baseline, 2))

In [ ]:
#Create ranges for possible  𝑝  and  𝑞  values. p_params should range between 0 and 25, by steps of 8. q_params should range between 0 and 3 by steps of 1.
p_params = range(0,25,8)
q_params = range(0,3,1) 

In [ ]:
list(p_params)
list(q_params)

In [ ]:
 for p in p_params:
        for q in q_params:
            order=(p,0,q)
            start_time=time.time()
            #train model
            model=ARIMA(y_train, order=order).fit()
            elapsed_time=round(time.time()-start_time,2)
            print(f"Trained ARIMA model {order} in {elapsed_time} seconds")
            #generate in-sample predictions
            y_pred=model.predict()
            #Calculate training MAE
            mae=mean_absolute_error(y_train, y_pred)
            print(mae)

In [ ]:
#Create empty dictionary for MAE values
mae_grid={}
for p in p_params:
    #create new key in dict with empty list
    mae_grid[p]=[]
    for q in q_params:
            #set the hyperparameters for the model
            order=(p,0,q)
            #start_timing
            start_time=time.time()
            #train model
            model=ARIMA(y_train, order=order).fit()
            #calculate elapsed time
            elapsed_time=round(time.time()-start_time,2)
            print(f"Trained ARIMA model {order} in {elapsed_time} seconds")
            #generate in-sample predictions
            y_pred=model.predict()
            #Calculate training MAE
            mae=mean_absolute_error(y_train, y_pred)
            #Add mae to dictionary
            mae_grid[p].append(mae)

In [ ]:
#Complete the code below to train a model with every combination of hyperparameters in p_params and q_params. Every time the model is trained, the mean absolute error is calculated and then saved to a dictionary. If you're not sure where to start, do the code-along with Nicholas!
# Create dictionary to store MAEs
mae_grid = dict()
# Outer loop: Iterate through possible values for `p`
for p in p_params:
    # Create key-value pair in dict. Key is `p`, value is empty list.
    mae_grid[p] = list()
    # Inner loop: Iterate through possible values for `q`
    for q in q_params:
        # Combination of hyperparameters for model
        order = (p, 0, q)
        # Note start time
        start_time = time.time()
        # Train model
        model = ARIMA(y_train, order=order).fit()
        # Calculate model training time
        elapsed_time = round(time.time() - start_time, 2)
        print(f"Trained ARIMA {order} in {elapsed_time} seconds.")
        # Generate in-sample (training) predictions
        y_pred = model.predict()
        # Calculate training MAE
        mae = mean_absolute_error(y_train, y_pred)
        # Append MAE to list in dictionary
        mae_grid[p].append(mae)
        
print(mae_grid)

In [ ]:
#Organize all the MAE's from above in a DataFrame names mae_df. Each row represents a possible value for  𝑞  and each column represents a possible value for  𝑝
mae_df= pd.DataFrame(mae_grid)
mae_df.round(4)

In [ ]:
#Create heatmap of the values in mae_grid. Be sure to label your x-axis "p values" and your y-axis "q values"
sns.heatmap(mae_df, cmap="Blues")
plt.xlabel("p values")
plt.ylabel("q values")
plt.title("ARMA Grid Search (Criterion:MAE)")

In [ ]:
#Use the plot_diagnostics method to check the residuals for your model. Keep in mind that the plot will represent the residuals from the last model you trained, so make sure it was your best model, too!
fig, ax = plt.subplots(figsize=(15, 12))
model.plot_diagnostics(fig=fig)

In [ ]:
#Complete the code below to perform walk-forward validation for your model for the entire test set y_test. Store your model's predictions in the Series y_pred_wfv. Choose the values for  𝑝  and  𝑞  that best balance model performance and computation time. Remember: This model is going to have to train 24 times before you can see your test MAE!
y_pred_wfv = pd.Series()
history = y_train.copy()
for i in range(len(y_test)):
    model = ARIMA(history, order=(8,0,1)).fit()
    next_pred = model.forecast()
    y_pred_wfv = y_pred_wfv.append(next_pred)
    history = history.append(y_test[next_pred.index])

In [ ]:
#First, generate the list of training predictions for your model. Next, create a DataFrame df_predictions with the true values y_test and your predictions y_pred_wfv (don't forget the index). Finally, plot df_predictions using plotly express. Make sure that the y-axis is labeled "P2"
df_predictions = pd.DataFrame({"y_test": y_test, "y_pred_wf":y_pred})
fig = px.line(df_predictions, labels={"value": "PM2.5"})
fig.show()